In [ ]:
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())


# cloning Detectron2

In [ ]:
!python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'     

# imports
cfg - choosing what model and dataset to refer.

Metadata - for adding segmentatin classes.

Visualizer - for plotting annotations above an image.

model_zoo - colection of nn models for different purposes.


In [ ]:
import csv
import cv2
import os
import random
import string

from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np

import detectron2
from detectron2.engine import DefaultPredictor, DefaultTrainer
from detectron2.config import get_cfg
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.utils.visualizer import ColorMode, Visualizer
from detectron2 import model_zoo
from detectron2.data.datasets import register_coco_instances
from detectron2.utils.logger import setup_logger

setup_logger()


# Data formating
we are feeding new dataset to an exsisting COCO dataset in Detectron2.

The COCO dataset is formatted in JSON and is a collection of

* “info”
* “licenses”
* “images”
* “annotations”
* “categories” 
* “segment info” for Panoptic annotations

In [ ]:
dataDir=Path('../input/sartorius-cell-instance-segmentation')
ids, masks=[],[]
#test_names = (dataDir/'test').ls()

# Register new dataset 
* fidding our sartorius dataset into coco-dataset.
* crossvalidationfold5 is a directory of json files that are in a compatible format for Detectron2
* access to our new dataset throw COCO (i.e dataset_dicts_train/valid , metadata_train)


In [ ]:
if 'my_dataset_train' in DatasetCatalog.list():
    DatasetCatalog.remove('my_dataset_train')

if "my_dataset_val" in DatasetCatalog.list():
    DatasetCatalog.remove("my_dataset_val")

register_coco_instances('my_dataset_train', {}, "../input/crossvalidationfold5/coco_cell_train_fold3.json", dataDir)
register_coco_instances("my_dataset_val", {}, "../input/crossvalidationfold5/coco_cell_valid_fold3.json", dataDir) 

dataset_dicts_train = DatasetCatalog.get("my_dataset_train")
dataset_dicts_valid = DatasetCatalog.get("my_dataset_val")
metadata_train = MetadataCatalog.get("my_dataset_train")


# Encode - Decode
**encoding RLE - a matrix mask to a tring:**
   * img: numpy array, 1 - mask, 0 - background
   * Returns run length as string formated
    
**decoding RLE - a tring to a mask matrix:**
   * mask_rle: run-length as string formated (start length)
   * shape: (height,width) of array to return 
   * Returns numpy array, 1 - mask, 0 - background

In [ ]:
def rle_encode(img):
    pixels = img.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    print(runs)
    runs[1::2] -= runs[::2]
    print(runs)
    return ' '.join(str(x) for x in runs)

def rle_decode(mask_rle, shape=(520, 704)):
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0]*shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    return img.reshape(shape)  # Needed to align to RLE direction

**To verify the data loading is correct, let's visualize the annotations of randomly selected samples in the training set:**

In [ ]:
for d in random.sample(dataset_dicts_train, 5):
    img = cv2.imread(d["file_name"])
    visualizer = Visualizer(img[:, :, ::-1], metadata=metadata_train, scale=1)
    out = visualizer.draw_dataset_dict(d)
    plt.figure(figsize=(10,10))
    plt.imshow(out.get_image())
    plt.show()

# Train the model
* set model configurations
* make a path to the output files
* train the model

In [ ]:
cfg = get_cfg()
THRESHOLDS = [.15, .35, .55]
MIN_PIXELS = [75, 150, 75]
# load model config

cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")
cfg.DATASETS.TRAIN = ("my_dataset_train",)
cfg.DATASETS.TEST = ("my_dataset_val",)
 

cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 3 
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.INPUT.MASK_FORMAT='bitmask'
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5
cfg.SOLVER.BASE_LR = 0.00025  # pick a good LR
cfg.SOLVER.WARMUP_ITERS = 10 #How many iterations to go from 0 to reach base LR

cfg.SOLVER.MAX_ITER = 2000
cfg.SOLVER.STEPS = (500, 1000)
cfg.TEST.DETECTIONS_PER_IMAGE = 1000
cfg.TEST.EVAL_PERIOD = 250
cfg.SOLVER.CHECKPOINT_PERIOD=250
cfg.MODEL.DEVICE = 'cuda'  #or "cpu"


In [ ]:
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = DefaultTrainer(cfg) 
trainer.resume_or_load(resume=False)
trainer.train()

# predictions
* update the model weights to our new traind weights.
* plot and save predictions to a csv file - "submission":
    1. ID      |    PREDICTION
    2. No overlaping masks
    3. RLE method



In [ ]:
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
predictor = DefaultPredictor(cfg)

In [ ]:
with open('submission.csv', 'w', newline = '') as f:
    writer = csv.writer(f)
    writer.writerow(['id', 'predicted'])
    test = '../input/sartorius-cell-instance-segmentation/test'
    for filename in os.listdir(test):
        img = cv2.imread(os.path.join(test,filename))
        img_id = filename
        predictions = predictor(img)
        
        visualizer = Visualizer(img[:, :, ::-1], metadata=metadata_train, scale=1, instance_mode=ColorMode.IMAGE_BW )
        out = visualizer.draw_instance_predictions(predictions["instances"].to("cpu"))
        plt.figure(figsize=(20,20))
        plt.imshow(out.get_image())
        plt.show()
        
        pred_class = torch.mode(predictions['instances'].pred_classes)[0]
        take = predictions['instances'].scores >= THRESHOLDS[pred_class]
        masks_list = predictions['instances'].pred_masks[take]
        masks_list = masks_list.cpu().numpy()

        used = np.zeros(img.shape[:2], dtype=int)

        for msk in masks_list:
            s= []
            msk = msk * (1-used)
            used += msk
            s = (rle_encode(msk))
            #print(s)
            writer.writerow([img_id, s])
    f.close()

